In [1]:
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import last,max

In [3]:
spark = SparkSession.builder.appName("dieseaseStream").getOrCreate()

In [4]:
line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
phone = spark.readStream.format("socket").option("host","192.168.1.142").option("port",9000).load()

In [5]:
ppoint = phone.select(explode(split(phone.value," ")).alias("id"))
ppoint = ppoint.withColumn('lat', split(ppoint["id"], '\,')[3].cast(types.DoubleType()))
ppoint = ppoint.withColumn('lon', split(ppoint["id"], '\,')[2].cast(types.DoubleType()))
ppoint = ppoint.withColumn('timestamp', split(ppoint["id"], '\,')[1].cast(types.LongType()))
ppoint = ppoint.withColumn('id', split(ppoint["id"], '\,')[0])
latestLat = ppoint.groupby('id').agg(max('timestamp'), last('lat'), last('lon')).toDF('id','time','lat','lon').drop('id','time').createOrReplaceTempView("latestLoc")

In [17]:
point = line.select(explode(split(line.value," ")).alias("lon"))
point = point.withColumn('lat', split(point["lon"], '\,')[1].cast(types.DoubleType()))
point = point.withColumn('lon', split(point["lon"], '\,')[0].cast(types.DoubleType()))
pointCount = point.groupBy('lat','lon').count().createOrReplaceTempView("outbreak")
query = spark.sql("""select count(*) from outbreak where 
outbreak.lat <= double((select lat from latestLoc) + 0.5) and
outbreak.lat >= double((select lat from latestLoc) - 0.5) and 
outbreak.lon <= double((select lon from latestLoc) + 0.5) and 
outbreak.lon >= double((select lon from latestLoc) - 0.5)""")

In [18]:
queryexe = query.writeStream.outputMode('complete').format('console').start()
queryexe.awaitTermination()
#query2 = latest.writeStream.outputMode('complete').format("console").start()
#spark.streams.awaitAnyTermination()

AnalysisException: 'Multiple streaming aggregations are not supported with streaming DataFrames/Datasets;;\nAggregate [count(1) AS count(1)#194L]\n+- Filter (((lat#176 <= cast((scalar-subquery#189 [] + cast(0.5 as double)) as double)) && (lat#176 >= cast((scalar-subquery#190 [] - cast(0.5 as double)) as double))) && ((lon#179 <= cast((scalar-subquery#191 [] + cast(0.5 as double)) as double)) && (lon#179 >= cast((scalar-subquery#192 [] - cast(0.5 as double)) as double))))\n   :  :- Project [lat#40]\n   :  :  +- SubqueryAlias `latestloc`\n   :  :     +- Project [lat#40, lon#41]\n   :  :        +- Project [id#19 AS id#38, max(timestamp)#31L AS time#39L, last(lat, false)#32 AS lat#40, last(lon, false)#33 AS lon#41]\n   :  :           +- Aggregate [id#19], [id#19, max(timestamp#14L) AS max(timestamp)#31L, last(lat#7, false) AS last(lat, false)#32, last(lon#10, false) AS last(lon, false)#33]\n   :  :              +- Project [split(id#5, \\,)[0] AS id#19, lat#7, lon#10, timestamp#14L]\n   :  